In [2]:
import os
import shutil
import librosa
from pydub import AudioSegment
import numpy as np
from openai import OpenAI
from tqdm import tqdm
import pandas as pd
import json
import pickle
import csv

c:\Users\qqq\anaconda3\envs\spyder-cf\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
def get_total_files(path):
    """递归计算路径下的所有文件数量"""
    total_files = 0
    for _, _, files in os.walk(path):
        total_files += len(files)
    return total_files


In [8]:
def get_volume_mark(file_name, start_time=None, end_time=None):
    def convert_mp3_to_wav(mp3_path):
        # 获取 MP3 文件的文件名（不包括路径）
        mp3_filename = os.path.basename(mp3_path)
        
        # 创建新的 WAV 文件名（在当前工作目录下）
        wav_filename = mp3_filename.replace('.mp3', '.wav').replace('.MP3', '.wav')
        
        # 如果文件已经在当前目录，则不需要复制
        if not os.path.exists(wav_filename):
            # 将 MP3 文件复制到当前工作目录
            shutil.copy2(mp3_path, mp3_filename)

        # 加载音频文件
        sound = AudioSegment.from_mp3(mp3_filename)

        # 导出为 WAV 格式
        wav_path = os.path.join(os.getcwd(), wav_filename)
        sound.export(wav_path, format="wav")
        
        if mp3_filename.endswith(".WAV"):
            return wav_path
        
        os.remove(mp3_filename)
        
        return wav_path

    # 加载音频文件
    mp3_file = file_name
    wav_file = convert_mp3_to_wav(mp3_file)
    
    # 使用 librosa 加载 wav 文件
    y, sr = librosa.load(wav_file)
    
    # 计算起始和结束样本索引
    if start_time is None:
        start_sample = 0
    else:
        start_sample = int(start_time * sr)
    
    if end_time is None:
        end_sample = len(y)
    else:
        end_sample = int(end_time * sr)
    
    # 截取音频片段
    y_segment = y[start_sample:end_sample]

    # 定义帧长和帧移
    frame_length = 2048  # 例如 2048 个样本点
    hop_length = 512     # 例如 512 个样本点

    # 计算每个帧的 RMS 值
    frames = librosa.util.frame(y_segment, frame_length=frame_length, hop_length=hop_length)
    rms_values = librosa.feature.rms(y=y_segment, frame_length=frame_length, hop_length=hop_length)[0]

    # 最大最小归一化
    min_rms = np.min(rms_values)
    max_rms = np.max(rms_values)
    normalized_rms = (rms_values - min_rms) / (max_rms - min_rms)

    # 将归一化后的 RMS 值乘以 100
    scaled_rms = normalized_rms * 100

    # 计算所有帧的平均值
    average_volume = np.mean(scaled_rms)

    # 清理临时文件
    os.remove(wav_file)

    return ("声音音量", float(average_volume))

In [36]:
volume_feature = get_volume_mark(R"D:\testcode  python\qipashuo\奇葩说第2季字幕音频\2-01-0_01_周思成\12月11日.WAV", start_time=10, end_time=20)
print(volume_feature)

('声音音量', 37.121559143066406)


In [10]:
def parse_timestamp(timestamp):
    h, m, s_ms = timestamp.split(':')
    s, ms = s_ms.split(',')
    return int(h), int(m), int(s), int(ms)

def total_milliseconds(h, m, s, ms):
    return h * 3600000 + m * 60000 + s * 1000 + ms

def calculate_interval(start_timestamp, end_timestamp):
    h1, m1, s1, ms1 = parse_timestamp(start_timestamp)
    h2, m2, s2, ms2 = parse_timestamp(end_timestamp)

    start_ms = total_milliseconds(h1, m1, s1, ms1)
    end_ms = total_milliseconds(h2, m2, s2, ms2)

    interval_ms = end_ms - start_ms
    return interval_ms

def calculate_wpm(text, duration_ms):
    word_count = len(text.strip())
    minutes = duration_ms / 60000.0
    wpm = word_count / minutes
    return wpm

In [11]:
def get_analysis(file_name):
    
    with open(file_name, 'r') as file:
        srt_content = file.read()

    # 解析SRT文件内容
    entries = srt_content.strip().split('\n\n')
    wpm_list = []

    for entry in entries:
        lines = entry.split('\n')
        if len(lines) >= 3:
            timestamps = lines[1]
            start_timestamp, end_timestamp = timestamps.split(' --> ')
            text = ' '.join(lines[2:])

            duration_ms = calculate_interval(start_timestamp, end_timestamp)
            wpm = calculate_wpm(text, duration_ms)
            wpm_list.append(wpm)
            
    avg_speed = sum(wpm_list) / len(wpm_list)
    
    return ("每分钟平均吐字", int(avg_speed))

In [12]:
import re


def time_to_seconds(time_str):
    """Convert SRT timestamp to seconds."""
    hours, minutes, seconds, milliseconds = map(int, re.split(r'[:,]', time_str))
    return hours * 3600 + minutes * 60 + seconds + milliseconds / 1000.0

def get_text_with_timestamps(file_path):
    # 打开并读取文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    subtitles = []
    buffer = []
    start_time = None
    end_time = None

    for line in lines:
        # 去除行末的换行符
        clean_line = line.strip()

        # 如果当前行为空行，则认为一组字幕结束
        if not clean_line:
            if buffer and start_time is not None and end_time is not None:
                # 将缓存中的字幕文本添加到结果列表中
                text = ' '.join(buffer)
                subtitles.append((text, start_time, end_time))
                buffer = []  # 清空缓存
                start_time = None
                end_time = None
        else:
            if '-->' in clean_line:
                # 提取起始时间和结束时间
                start_time_str, end_time_str = clean_line.split(' --> ')
                start_time = time_to_seconds(start_time_str)
                end_time = time_to_seconds(end_time_str)
            elif '0' <= clean_line[0] <= '9':
                continue
            else:
                buffer.append(clean_line)

    # 添加最后一组字幕（如果有的话）
    if buffer and start_time is not None and end_time is not None:
        text = ' '.join(buffer)
        subtitles.append((text, start_time, end_time))

    return subtitles

# 示例用法
file_path = R"D:\testcode  python\qipashuo\奇葩说第2季字幕音频\2-01-0_01_周思成\12月11日.srt"
subtitles = get_text_with_timestamps(file_path)
# for subtitle in subtitles:
#    print(subtitle)


def get_split(text_list):
    client = OpenAI(api_key="sk-51d2985a4cd14eba9709a9bf31ad0930", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    
    # 构建prompt
    prompt = """
    请根据提供的字幕元组列表，将连续出现的句子组合成一段完整的话，并标记为辩论中的立论、陈述或证据。
    每个段落应包含文本、类型（立论、陈述、证据）、起始时间和结束时间。以下是元组列表的格式：

    [(句子, 起始时间, 结束时间)]

    请按照以下格式输出结果，注意不要输出其他内容，并确保所有的句子都被考虑到：
    [(文本, 类型, 起始时间, 结束时间), ...]
    """
    
    response = client.chat.completions.create(
        model="qwen-plus-1127",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"{text_list}"},
        ],
        stream=False
        )
    
    # print(response.choices[0].message.content)
    return response.choices[0].message.content


print(get_split(subtitles))

[("参加了奇葩说，但是实力不够被淘汰了。别人来问我说，哎呀你怎么会被淘汰。我出于面子，我说因为他们太奇葩了，我做不到。", "陈述", 0.1, 12.4),
("所以呢我就举了一些例子来例证他们太奇葩，放到我的电台节目里面。结果呢虽然我实力不够，但是粉丝还是有一些的，所以很多人就跑去跟潇潇说，哎呀那个人在暗讽你。其实我没有提到名字。", "证据", 13.066, 26.866),
("于是我们俩共同的好朋友刘思达就跑过来跟我说，你怎么可以这样说潇潇，潇潇那么棒。其实我没有提到你的名字，但是呢还是对对你表示非常的抱歉。", "陈述", 27.166, 39.133),
("当好朋友有难的时候，我们必须挺身而出，虽然我们不需要挺到说你来踢我好了，但是我们必须要及时如实的告诉他。", "立论", 42.6, 53.7),
("比如说我们俩去喝下午茶，起来以后，我发现他的后面红红的。哈哈哈他是猴吗？哈哈哈什么叫后面红红的？你懂的。他的后面红红的，我好羞羞。", "陈述", 53.7, 69.7),
("我告诉还是不告诉？哦他懂了。哦懂了。哦他懂了。懂了我告诉还是不告诉？又不是我红红的，关我什么事。好吓人。", "陈述", 70.4, 82.566),
("但是我不告诉他，在路上走的时候，路人会看见，好朋友会看见，甚至敌人会拍照发朋友圈，我还是人吗？我是他的好朋友。", "证据", 83.533, 95.766),
("所以，如果你希望你的好朋友也这样诚信，讲义气的话，请你告诉他谢谢。", "立论", 96.0, 102.666)]


In [13]:
def get_interaction_mark(text):
    client = OpenAI(api_key="sk-51d2985a4cd14eba9709a9bf31ad0930", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    details = ["情感：从文字的情感，幽默性，情感流露等方面考察", "交流性：从提供建议，提问，自我表达等方面考察", "凝聚力：从称呼、社交性话语使用等方面考察"]
    response = client.chat.completions.create(
        model="qwen-plus-1127",
        messages=[
            {"role": "system", "content": f"你是一个辩论手，需要根据用户提供的辩论稿,对社会临场感其打分，综合情感，交流性，凝聚力三种标准， {details}, 仅仅回复0-10之间的一个数字,不要回复分析过程。"},
            {"role": "user", "content": text},
        ],
        stream=False
        )
    
    # print(response.choices[0].message.content)
    return [("社会临场感", response.choices[0].message.content)]

In [14]:
def get_text_from_srt(file_path):
    # 打开并读取文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    text = []
    buffer = []

    for line in lines:
        # 去除行末的换行符
        clean_line = line.strip()

        # 如果当前行为空行，则认为一组字幕结束
        if not clean_line:
            if buffer:
                # 将缓存中的字幕文本添加到结果列表中
                text.append(' '.join(buffer))
                buffer = []  # 清空缓存
        else:
            if '0' <= clean_line[0] <= '9':
                continue
              
            buffer.append(clean_line)

    # 添加最后一组字幕（如果有的话）
    if buffer:
        text.append(' '.join(buffer))

    return '\n'.join(text)

In [15]:
def get_quality_mark(text):
    client = OpenAI(api_key="sk-51d2985a4cd14eba9709a9bf31ad0930", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    prompt = """
    请根据提供的辩论稿，对质量进行打分，综合相关性、可接受性和充分性标准。每个标准的详细描述如下：
    - 相关性标准: 如果一个论点的所有前提都支持该主张的真实性(或虚假性)，则该论点满足相关性标准。
    - 可接受性标准: 如果一个论点的前提代表了无可争议的常识或事实，那么它就符合可接受性标准。
    - 充分性标准: 如果一个论证的前提提供了足够的证据来接受或拒绝该主张，则该论点就符合充分性标准。

    请按照以下格式返回结果：
    [(标准名称, 打分), ...]

    其中，打分为0到10之间的整数，注意不要返回其他内容
    """

    response = client.chat.completions.create(
        model="qwen-plus-1127",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": text},
        ],
        stream=False
    )

    # 获取AI的响应
    result = response.choices[0].message.content.strip()

    # 解析结果
    try:
        return eval(result)
    except Exception as e:
        print(f"Error parsing response: {e}")
        print(f"Response content: {result}")
        return []

# 示例调用
text = """我认为这是正确的观点 因为有足够的数据支持 并且专家也认同这一点"""
judge_result = get_quality_mark(text)
print(judge_result)

[('相关性', 8), ('可接受性', 7), ('充分性', 6)]


In [16]:
timestamp_df = pd.read_excel("D:\\testcode  python\\qipashuo\\时间戳_完整版.xlsx")
timestamp_df.head()

,Unnamed: 0,序号,排期,辩题,环节,辩手姓名,时间戳,备注,季度,timestamps
0,0,1,2-01-0,好友恋人出轨 要不要告诉TA,正方,周思成,24：00-25：45,以下为第二季内容,2,1440-1545
1,1,2,2-01-0,好友恋人出轨 要不要告诉TA,反方,肖骁,26：10-29：30,第二季视频上线中没有写明第几期，而是按播出日期命名，以第二季上线最早的一期节目为第一期,2,1570-1770
2,2,3,2-01-0,好友恋人出轨 要不要告诉TA,正方,李如儒,30：20-33：17,不是组队的方式陈述观点，因此只分正反方，没有一辩二辩三辩环节,2,1820-1997
3,3,4,2-01-0,好友恋人出轨 要不要告诉TA,反方,范湉湉,34：25-36：30,NaN,2,2065-2190
4,4,5,2-01-0,好友恋人出轨 要不要告诉TA,正方,小K,37：10-38：35,NaN,2,2230-2315


In [17]:
def parse_filename(filename):
    return filename.split("_")[0], filename.split("_")[-1]

def find_row(date, name):
    global timestamp_df
    for _, row in timestamp_df.iterrows():
        if row["排期"] == date and row["辩手姓名"] == name:
            return row["timestamps"].split("-")
        
filename = "2-03-0_01_大兵"
a, b = parse_filename(filename)
print(find_row(a, b))

['960', '1200']


In [52]:
def process_srt(root, final_feature, bullet_path, season):
    global audio_files, srt_files
    
    file_path = srt_files.get(root, 0)
    
    if not file_path:
        return 
    
    text = get_text_from_srt(file_path)
    final_feature[root]["辩论原稿"] = text
    
    text_list = get_text_with_timestamps(file_path)
    
    try:
        text_list = eval(get_split(text_list))
    except Exception as e:
        print(f"Error processing text_list for {file_path}: {e}")
        return 
    
    audio_file = audio_files.get(root, 0)
    
    for idx, text_tp in enumerate(text_list):
        text = text_tp[0]
        feature = []
        
        try:
            interaction_mark = get_interaction_mark(text)
            feature.append(("社会临场感得分", interaction_mark))
        except Exception as e:
            print(f"Error processing interaction mark for {file_path}: {e}")
            
        try:
            quality = get_quality_mark(text)
            feature.append(("辩论质量得分", quality))
        except Exception as e:
            print(f"Error processing quality mark for {file_path}: {e}")
            
        try:
            speed = int(len(text_tp[0])/(text_tp[3] - text_tp[2] + 1))
            feature.append(("平均吐字速度", speed))
        except Exception as e:
            print(f"Error processing speed mark for {file_path}: {e}")
            
        if audio_file:
            print(audio_file)
            try:
                volume = get_volume_mark(audio_file, int(text_tp[2]), int(text_tp[3]))
                feature.append(("辩论音量计算值", volume))
            except Exception as e:
                print(f"Error processing volume mark for {audio_file}: {e}")
                
        # 读取起始时间戳，加上偏移量
        date, name = parse_filename(root.split("\\")[-1])
        st, ed = map(int, find_row(date, name))
        bullet_name = os.path.join(bullet_path, f"{date.split('-')[1]}.json")
        print(bullet_name)
        ls = json.load(open(bullet_name, "r", encoding="utf-8"))
        pos, neg, tot = 0, 0, 0
        for [sentiment, timestamp] in ls:
            timestamp = int(timestamp)
            print(text_tp)
            if timestamp >= st + int(text_tp[2]) and timestamp <= st + int(text_tp[3]):
                if sentiment == "Positive":
                    pos += 1
                elif sentiment == "Negative":
                    neg += 1
                tot += 1
            else: 
                break
        feature.append(("情感分析", f"积极个数：{pos}", f"消极个数：{neg}", f"弹幕总数：{tot}", f"极性计算值：{(pos - neg) / (tot + 1)}"))
        print(feature)
        final_feature[root][f"adu{idx}"] = feature
        with open(f"{season}\\{date}-{name}.txt", "a", encoding="utf-8") as f:
            f.write(json.dumps(feature, ensure_ascii=False) + "\n")

In [53]:
from collections import defaultdict


path = R"D:\testcode  python\qipashuo\奇葩说第2季字幕音频"
season = "season2"
bullet_path = R"D:\testcode  python\qipashuo\bullet_season2"
error_files = []
total = get_total_files(path)
pbar = tqdm(total=total, desc='Processing', unit='file')
final_feature = defaultdict(dict)


# 分类存储文件路径
srt_files = dict()
audio_files = dict()

for root, dirs, files in os.walk(path):
    for file in files:
        file_path = os.path.join(root, file)
        if file.endswith(".srt"):
            srt_files[root] = os.path.join(root, file)
        elif file.endswith(".wav") or file.endswith(".mp3") or file.endswith(".MP3") or file.endswith(".WAV"):
            audio_files[root] = os.path.join(root, file)
            
# 保留srt，确保先处理出adu
# print(srt_files)
for x in srt_files.keys():
    print(x.split("\\")[-1])
    process_srt(x, final_feature, bullet_path, season)
    break

# # 先处理 SRT 文件
# for root, file in srt_files:
#     pbar.update(1)
#     root_folder = root.split("\\")[-1]  # 使用反斜杠分割路径
#     file_path = os.path.join(root, file)
#     process_srt(file_path, final_feature, root_folder)

# # 再处理 音频文件
# for root, file in audio_files:
#     pbar.update(1)
#     root_folder = root.split("\\")[-1]  # 使用反斜杠分割路径
#     file_path = os.path.join(root, file)
#     process_audio(file_path, final_feature, root_folder)

Processing:   0%|          | 0/526 [03:16<?, ?file/s]


2-01-0_01_周思成
D:\testcode  python\qipashuo\奇葩说第2季字幕音频\2-01-0_01_周思成\12月11日.WAV
D:\testcode  python\qipashuo\bullet_season2\01.json
('参加了奇葩说，但是实力不够被淘汰了。别人来问我说，哎呀你怎么会被淘汰。我出于面子，我说因为他们太奇葩了，我做不到。', '陈述', 0.1, 12.4)
[('社会临场感得分', [('社会临场感', '3')]), ('辩论质量得分', [('相关性', 4), ('可接受性', 3), ('充分性', 2)]), ('平均吐字速度', 4), ('辩论音量计算值', ('声音音量', 37.584537506103516)), ('情感分析', '积极个数：0', '消极个数：0', '弹幕总数：0', '极性计算值：0.0')]
D:\testcode  python\qipashuo\奇葩说第2季字幕音频\2-01-0_01_周思成\12月11日.WAV
D:\testcode  python\qipashuo\bullet_season2\01.json
('所以呢我就举了一些例子来例证他们太奇葩，放到我的电台节目里面。结果呢虽然我实力不够，但是粉丝还是有一些的，所以很多人就跑去跟潇潇说，哎呀那个人在暗讽你。其实我没有提到名字。', '证据', 13.066, 26.866)
[('社会临场感得分', [('社会临场感', '3')]), ('辩论质量得分', [('相关性', 3), ('可接受性', 4), ('充分性', 2)]), ('平均吐字速度', 5), ('辩论音量计算值', ('声音音量', 38.343963623046875)), ('情感分析', '积极个数：0', '消极个数：0', '弹幕总数：0', '极性计算值：0.0')]
D:\testcode  python\qipashuo\奇葩说第2季字幕音频\2-01-0_01_周思成\12月11日.WAV
D:\testcode  python\qipashuo\bullet_season2\01.json
('于是我们俩共同的好朋友刘思达就跑过来跟我说，你怎么可以这样说潇潇，潇潇那么棒。其实我没有提到你的名字，但是

In [54]:
final_feature

defaultdict(dict,
            {'D:\\testcode  python\\qipashuo\\奇葩说第2季字幕音频\\2-01-0_01_周思成': {'辩论原稿': '参加了奇葩说\n但是实力不够被淘汰了\n别人来问我说\n哎呀你怎么会被淘汰\n我出于面子\n我说因为他们太奇葩了\n我做不到\n所以呢我就举了一些例子来例证\n他们太奇葩\n放到我的电台节目里面\n结果呢虽然我实力不够\n但是粉丝还是有一些的\n所以很多人就跑去跟潇潇说\n哎呀那个人在暗讽你\n其实我没有提到名字\n于是我们俩共同的好朋友\n刘思达就跑过来跟我说\n你怎么可以这样说潇潇\n潇潇那么棒\n其实我没有提到你的名字\n但是呢还是对对你表示非常的抱歉\n你的发言时间快到了啊\n当好朋友有难的时候\n我们必须挺身而出\n虽然我们不需要挺到说你来踢我好了\n但是我们必须要及时如实的告诉他\n比如说我们俩去喝下午茶\n起来以后\n我发现他的后面红红的\n哈哈哈他是猴吗\n哈哈哈什么叫后面红红的\n你懂的\n他的后面红红的\n我好羞羞\n我告诉还是不告诉\n哦他懂了\n哦懂了\n哦他懂了\n懂了我告诉还是不告诉\n又不是我红红的\n关我什么事\n好吓人\n但是我不告诉他\n在路上走的时候\n路人会看见\n好朋友会看见\n甚至敌人会拍c照发朋友圈\n我还是人吗\n我是他的好朋友\n所以\n如果你希望你的好朋友也这样诚信\n讲义气的话\n请你告诉他谢谢',
              'adu0': [('社会临场感得分', [('社会临场感', '3')]),
               ('辩论质量得分', [('相关性', 4), ('可接受性', 3), ('充分性', 2)]),
               ('平均吐字速度', 4),
               ('辩论音量计算值', ('声音音量', 37.584537506103516)),
               ('情感分析', '积极个数：0', '消极个数：0', '弹幕总数：0', '极性计算值：0.0')],
              'adu1': [('社会临场感得分', [('社会临场感', '3')]),
               ('辩论质量得分', [('相关性

In [ ]:
# 指定要保存的文件路径
output_file = 'final_feature.pkl'

# 使用 with 语句打开文件，并使用 pickle.dump() 方法将数据写入文件
with open(output_file, 'wb') as file:
    pickle.dump(final_feature, file)

print(f"Data has been saved to {output_file}")

# 收集所有的列名
column_names = set()
for inner_dict in final_feature.values():
    column_names.update(inner_dict.keys())
# 添加'name'列
column_names = ['name'] + sorted(column_names)

# 准备数据行
rows = []
for name, inner_dict in final_feature.items():
    row = [name] + [inner_dict.get(col, '') for col in column_names[1:]]
    rows.append(row)

# 写入CSV文件
output_file = 'output.csv'
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入表头
    writer.writerow(column_names)
    # 写入数据行
    writer.writerows(rows)

print(f"Data has been written to {output_file}")

with open(f"./{season}/error.txt", "w") as f:
  for x in error_files:
    f.write(str(x)+"\n")

In [7]:
import pandas as pd


df = pd.read_csv('output.csv')
# list(df.columns)
ls = sorted(list(df.columns)[1:-1], key=lambda x: int(x[3:]))
ls = ["name"] + ls + ["辩论原稿"]
ls

['name',
 'adu0',
 'adu1',
 'adu2',
 'adu3',
 'adu4',
 'adu5',
 'adu6',
 'adu7',
 'adu8',
 'adu9',
 'adu10',
 'adu11',
 'adu12',
 'adu13',
 'adu14',
 'adu15',
 'adu16',
 'adu17',
 'adu18',
 'adu19',
 'adu20',
 'adu21',
 '辩论原稿']

In [10]:
df_reordered = df[ls]
for i in range(len(df)):
    df_reordered.loc[i, 'name'] = df.loc[i, 'name'].split("/")[-1]
df_reordered.sort_values(by='name', inplace=True)
df_reordered.to_csv('output1.csv', index=False)